# Relative Strength Index

* It measures the force of a share in negotiation by analyzing the changes on close prices
* It is an indicator / trend tracker of **oscillator** type



* It isn't based on **moving averages**
  * Indicators based on moving averages tend to emit trading signals **after** a event occurs (reversion)
  * **Oscillation** indicators emit trading signals **before** the reversions

* The RSI indicator can emit signals **before** or **at** the reversions, but never **after** them


$$
RSI = 100-\frac{100}{1 + FR}
$$

$$
FR = \frac{avg(wins)}{avg(losses)}
$$

The typical window for the moving averages is $14$ days

These averages are calculated by

* *First data point is* $sum(last \ n \ periods) / n$ where typically $n=14$
* *Following data points are* $\frac{(last \ value) * (n-1) \ + \ (actual \ value)}{n} $ where typically $n=14$

The resultant values will always be between $0$ and $100$





## RSI interpretation

* At a maximum **peak** the price must revert by *falling* (top on prices)
* At a minimum **trough** the price must revert by *raising* (bottom on prices)



Regions of **overselling** and **overbuying** indicates that the prices are too expensive (and will fall) or too cheap (and will raise) 

* Generally (depending on the share) the region of:
  * **Overselling** stays between $70$ and $100$
  * **Overbuying** stays between $0$ and $30$



In [1]:
import sys
import os

import chart_studio.plotly as plty
import plotly.graph_objs as gobjs

import plotly

sys.path.insert(0, os.path.abspath('../py'))
from Secrets import ReadSecrets


account = ReadSecrets()\
        .set_secrets_path('../secrets.json')\
        .set_default_accessors()\
        .access('plotly')['main_account']

plty.sign_in(account['username'], account['apiKey'])


from datetime import datetime as calendar
import numpy as np

from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
yf.pdr_override()

from DataHelper import DataHelper

from plotly import tools

/home/andre/Desktop/pythonTradingAlgos/tradingalgos/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
data, close, extractors = DataHelper.get_history_formatted(['AAPL', 'MSFT', '^GSPC'], calendar(2016, 1 ,1), calendar(2020, 2, 1))
msft = extractors['MSFT']()
aapl = extractors['AAPL']()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
window = 14

In [4]:
rsi = pd.DataFrame(index=aapl.index)

# get the percentual changes between day[i] and day[i-1]
rsi_changes = aapl.Close.diff(1)

# get days with positive profit (gain)
rsi['gain'] = rsi_changes[rsi_changes > 0]

# get days with negative profit (loss)
rsi['loss'] = rsi_changes[rsi_changes < 0] * (-1)

# gain and loss mean
rsi['gainAvg'] = np.NaN
rsi['lossAvg'] = np.NaN

# get the mean for "window" period
rsi.gainAvg[window] = rsi.iloc[0:window].gain.mean()
rsi.lossAvg[window] = rsi.iloc[0:window].loss.mean()

rsi = rsi[window:].fillna(0.0)

# apply the formula
for i in range(1, len(rsi)):
    rsi.gainAvg[i] = (rsi.gainAvg[i-1] * (window -1) + rsi.gain[i]) / window
    rsi.lossAvg[i] = (rsi.lossAvg[i-1] * (window -1) + rsi.loss[i]) / window
        
rsi['value'] = 100 - (100 / (1 + (rsi.gainAvg / rsi.lossAvg )))

In [5]:
trace_rsi = gobjs.Scatter(
                    x=rsi.index,
                    y=rsi.value,
                    xaxis='x2',
                    yaxis='y2')


trace_underpriced = gobjs.Scatter(
                x=rsi.index,
                y=np.where(rsi.value < 30, rsi.value, np.NaN),
                name = "Overpriced",
                line = dict(color = '#22B222'),
                opacity = 1,
                xaxis='x2',
                yaxis='y2',
            )

trace_overpriced = gobjs.Scatter(
                x=rsi.index,
                y=np.where(rsi.value > 70, rsi.value, np.NaN),
                name = "Overpriced",
                line = dict(color = '#B22222'),
                opacity = 1,
                xaxis='x2',
                yaxis='y2',
            )

trace_candles = gobjs.Candlestick(x=aapl.index,
                    open=aapl.Open,
                    high=aapl.High,
                    low=aapl.Low,
                    close=aapl.Close)


data = [trace_rsi, trace_overpriced, trace_underpriced, trace_candles]

layout = gobjs.Layout(
    xaxis=dict(
        domain=[0, 1],
        rangeslider={"visible": False},
    ),
    yaxis=dict(
        domain=[0.75, 1],        
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='y2'
    ),
    yaxis2=dict(
        domain=[0, 0.45],
        anchor='x2',
        range=[0, 100]
        
    ),
        shapes=[
            # Line Horizontal
            {
                'xref': 'x2',
                'yref': 'y2',
                'type': 'line',
                'x0': aapl.index[0],
                'y0': '30',
                'x1': aapl.index[-1],
                'y1': '30',
                'line': {
                    'color': 'rgba(200, 200, 200, .5)',
                    'width': 2,
                    'dash': 'dashdot',
                },
            },
            {
                'xref': 'x2',
                'yref': 'y2',
                'type': 'line',
                'x0': aapl.index[0],
                'y0': '70',
                'x1': aapl.index[-1],
                'y1': '70',
                'line': {
                    'color': 'rgba(200, 200, 200, .5)',
                    'width': 2,
                    'dash': 'dashdot',
                },
            }        
        ]        
    
)
fig = gobjs.Figure(data=data, layout=layout)

plty.iplot(fig)